# **Amazon Lookout for Equipment** - Demonstration on an anonymized compressor dataset
*Part 5: Scheduling regular inference calls*

## Initialization
---
In this notebook, we will update the repository structure to add an inference directory in the data folder:
```
/lookout-equipment-demo/getting_started/
|
├── data/
|   |
|   ├── inference-data/
|   |   ├── input/
|   |   └── output/
|   |
|   ├── labelled-data/
|   |   └── labels.csv
|   |
|   └── training-data/
|       └── expander/
|           ├── subsystem-01
|           |   └── subsystem-01.csv
|           |
|           ├── subsystem-02
|           |   └── subsystem-02.csv
|           |
|           ├── ...
|           |
|           └── subsystem-24
|               └── subsystem-24.csv
|
├── dataset/                                <<< Original dataset <<<
|   ├── labels.csv
|   ├── tags_description.csv
|   ├── timeranges.txt
|   └── timeseries.zip
|
├── notebooks/
|   ├── 1_data_preparation.ipynb
|   ├── 2_dataset_creation.ipynb
|   ├── 3_model_training.ipynb
|   ├── 4_model_evaluation.ipynb
|   ├── 5_inference_scheduling.ipynb        <<< This notebook <<<
|   └── config.py
|
└── utils/
    ├── aws_matplotlib_light.py
    └── lookout_equipment_utils.py
```

In [ ]:
!pip install --quiet --upgrade tqdm sagemaker

In [ ]:
import boto3
import config
import datetime
import os
import pandas as pd
import pprint
import pyarrow as pa
import pyarrow.parquet as pq
import sagemaker
import s3fs
import sys
import time
import uuid

# Helper functions for managing Lookout for Equipment API calls:
sys.path.append('../utils')
import lookout_equipment_utils as lookout

### Parameters

In [ ]:
DATA       = os.path.join('..', 'data')
RAW_DATA   = os.path.join('..', 'dataset')
INFER_DATA = os.path.join(DATA, 'inference-data')

os.makedirs(os.path.join(INFER_DATA, 'input'), exist_ok=True)
os.makedirs(os.path.join(INFER_DATA, 'output'), exist_ok=True)

ROLE_ARN         = sagemaker.get_execution_role()
REGION_NAME      = boto3.session.Session().region_name
DATASET_NAME     = config.DATASET_NAME
BUCKET           = config.BUCKET
PREFIX_TRAINING  = config.PREFIX_TRAINING
PREFIX_LABEL     = config.PREFIX_LABEL
PREFIX_INFERENCE = config.PREFIX_INFERENCE
MODEL_NAME       = config.MODEL_NAME

### Scheduler configuration


In [ ]:
# Name of the model on which you want to create this inference scheduler
MODEL_NAME_FOR_CREATING_INFERENCE_SCHEDULER = MODEL_NAME

# Name of the inference scheduler you want to create
INFERENCE_SCHEDULER_NAME = config.INFERENCE_SCHEDULER_NAME

# Mandatory parameters:
INFERENCE_DATA_SOURCE_BUCKET = BUCKET
INFERENCE_DATA_SOURCE_PREFIX = f'{PREFIX_INFERENCE}/input/'
INFERENCE_DATA_OUTPUT_BUCKET = BUCKET
INFERENCE_DATA_OUTPUT_PREFIX = f'{PREFIX_INFERENCE}/output/'
ROLE_ARN_FOR_INFERENCE = ROLE_ARN
DATA_UPLOAD_FREQUENCY = 'PT5M'

#### Optional parameters


In [ ]:
DATA_DELAY_OFFSET_IN_MINUTES = None
INPUT_TIMEZONE_OFFSET = '+00:00'
COMPONENT_TIMESTAMP_DELIMITER = '_'
TIMESTAMP_FORMAT = 'yyyyMMddHHmmss'

### Create the inference scheduler


In [ ]:
scheduler = lookout.LookoutEquipmentScheduler(
    scheduler_name=INFERENCE_SCHEDULER_NAME,
    model_name=MODEL_NAME_FOR_CREATING_INFERENCE_SCHEDULER,
    region_name=REGION_NAME
)

scheduler_params = {
    'input_bucket': INFERENCE_DATA_SOURCE_BUCKET,
    'input_prefix': INFERENCE_DATA_SOURCE_PREFIX,
    'output_bucket': INFERENCE_DATA_OUTPUT_BUCKET,
    'output_prefix': INFERENCE_DATA_OUTPUT_PREFIX,
    'role_arn': ROLE_ARN_FOR_INFERENCE,
    'upload_frequency': DATA_UPLOAD_FREQUENCY,
    'delay_offset': DATA_DELAY_OFFSET_IN_MINUTES,
    'timezone_offset': INPUT_TIMEZONE_OFFSET,
    'component_delimiter': COMPONENT_TIMESTAMP_DELIMITER,
    'timestamp_format': TIMESTAMP_FORMAT
}

scheduler.set_parameters(**scheduler_params)

## Prepare the inference data


In [ ]:
# Let's load all our original signals:
all_tags_fname = os.path.join(DATA, 'training-data', 'expander.parquet')
table = pq.read_table(all_tags_fname)
all_tags_df = table.to_pandas()
del table
all_tags_df.head()

In [ ]:
tags_description_fname = os.path.join(RAW_DATA, 'tags_description.csv')
tags_description_df = pd.read_csv(tags_description_fname)
components = tags_description_df['Subsystem'].unique()
tags_description_df.head()

In [ ]:
start = pd.to_datetime('2015-11-21 04:00:00')

In [ ]:
# How many sequences do we want to extract:
num_sequences = 12

# The scheduling frequency in minutes: this **MUST** match the
# resampling rate used to train the model:
frequency = 5

# Loops through each sequence:
for i in range(num_sequences):
    end = start + datetime.timedelta(minutes=+frequency - 1)
    
    # Rounding time to the previous 5 minutes:
    tm = datetime.datetime.now()
    tm = tm - datetime.timedelta(
        minutes=tm.minute % frequency,
        seconds=tm.second,
        microseconds=tm.microsecond
    )
    tm = tm + datetime.timedelta(minutes=+frequency * (i))
    current_timestamp = (tm).strftime(format='%Y%m%d%H%M%S')

    # For each sequence, we need to loop through all components:
    print(f'Extracting data from {start} to {end}')
    new_index = None
    for component in components:
        # Extracting the dataframe for this component and this particular time range:
        signals = list(tags_description_df.loc[(tags_description_df['Subsystem'] == component), 'Tag'])
        signals_df = all_tags_df.loc[start:end, signals]
        
        # We need to reset the index to match the time 
        # at which the scheduler will run inference:
        if new_index is None:
            new_index = pd.date_range(
                start=tm,
                periods=signals_df.shape[0], 
                freq='1min'
            )
        signals_df.index = new_index
        signals_df.index.name = 'Timestamp'
        signals_df = signals_df.reset_index()

        # Export this file in CSV format:
        component_fname = os.path.join(INFER_DATA, 'input', f'{component}_{current_timestamp}.csv')
        signals_df.to_csv(component_fname, index=None)
    
    start = start + datetime.timedelta(minutes=+frequency)
    
# Upload the whole folder to S3, in the input location:
INFERENCE_INPUT = os.path.join(INFER_DATA, 'input')
!aws s3 cp --recursive --quiet $INFERENCE_INPUT s3://$BUCKET/$PREFIX_INFERENCE/input

In [ ]:
# Now that we've prepared the data, create the scheduler by running:
create_scheduler_response = scheduler.create()

## Get inference results
---

### List inference executions

In [ ]:
START_TIME_FOR_INFERENCE_EXECUTIONS = None
END_TIME_FOR_INFERENCE_EXECUTIONS = None
EXECUTION_STATUS = None

execution_summaries = []

while len(execution_summaries) == 0:
    execution_summaries = scheduler.list_inference_executions(
        start_time=START_TIME_FOR_INFERENCE_EXECUTIONS,
        end_time=END_TIME_FOR_INFERENCE_EXECUTIONS,
        execution_status=EXECUTION_STATUS
    )
    if len(execution_summaries) == 0:
        print('WAITING FOR THE FIRST INFERENCE EXECUTION')
        time.sleep(60)
        
    else:
        print('FIRST INFERENCE EXECUTED\n')
        break
            
execution_summaries

### Download inference results
Let's have a look at the content now available in the scheduler output location: each inference execution creates a subfolder in the output directory. The subfolder name is the timestamp (GMT) at which the inference was executed and it contains a single [JSON lines](https://jsonlines.org/) file named `results.jsonl`:

In [ ]:
# Fetch the list of execution summaries in case all executions were not captured yet:
execution_summaries = scheduler.list_inference_executions()
execution_summaries[0]

In [ ]:
# Loops through the executions summaries:
results_json = []
for execution_summary in scheduler.execution_summaries:
    print('.', end='')
    
    # We only get an output if the inference execution is a sucess:
    status = execution_summary['Status']
    if status == 'SUCCESS':
        # Download the JSON-line file locally:
        bucket = execution_summary['CustomerResultObject']['Bucket']
        key = execution_summary['CustomerResultObject']['Key']
        current_timestamp = key.split('/')[-2]
        local_fname = os.path.join(INFER_DATA, 'output', f'centrifugal-pump_{current_timestamp}.jsonl')
        s3_fname = f's3://{bucket}/{key}'
        
        !aws s3 cp --quiet $s3_fname $local_fname
        
        # Opens the file and concatenate the results into a dataframe:
        with open(local_fname, 'r') as f:
            content = [eval(line) for line in f.readlines()]
            results_json = results_json + content
    
# Build the final dataframes with all the results:
results_df = pd.DataFrame(results_json)
results_df['timestamp'] = pd.to_datetime(results_df['timestamp'])
results_df = results_df.set_index('timestamp')
results_df = results_df.sort_index()
results_df.head()

The content of each JSON lines file follows this format:
    
```json
[
    {
        'timestamp': '2021-04-17T13:25:00.000000',
        'prediction': 1,
        'prediction_reason': 'ANOMALY_DETECTED',
        'diagnostics': [
            {'name': 'subsystem-19\\signal-067', 'value': 0.12},
            {'name': 'subsystem-18\\signal-099', 'value': 0.0},
            {'name': 'subsystem-09\\signal-016', 'value': 0.0},
                                  .
                                  .
                                  .
            {'name': 'subsystem-06\\signal-119', 'value': 0.08},
            {'name': 'subsystem-10\\signal-071', 'value': 0.02},
            {'name': 'subsystem-20\\signal-076', 'value': 0.02}
        ]
    }
    ...
]
```


### Visualizing the inference results

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np

%matplotlib inline

# Load style sheet:
plt.style.use('../utils/aws_matplotlib_light.py')

# Get colors from custom AWS palette:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

#### Single inference analysis
Let's first expand the results to expose the content of the **diagnostics** column above into different dataframe columns:

In [ ]:
expanded_results = []
for index, row in results_df.iterrows():
    new_row = dict()
    new_row.update({'timestamp': index})
    new_row.update({'prediction': row['prediction']})
    
    if row['prediction'] == 1:
        diagnostics = pd.DataFrame(row['diagnostics'])
        diagnostics = dict(zip(diagnostics['name'], diagnostics['value']))
        new_row = {**new_row, **diagnostics}
        
    expanded_results.append(new_row)
    
expanded_results = pd.DataFrame(expanded_results)
expanded_results['timestamp'] = pd.to_datetime(expanded_results['timestamp'])
expanded_results = expanded_results.set_index('timestamp')
expanded_results.head()

Each detected event have some detailed diagnostics. Let's unpack the details for the first event and plot a similar bar chart than what the console provides when it evaluates a trained model:

In [ ]:
event_details = pd.DataFrame(expanded_results.iloc[0, 1:]).reset_index()
event_details.columns = ['name', 'value']
event_details = event_details.sort_values(by='value')
event_details_limited = event_details.tail(10)

# We can then plot a horizontal bar chart:
y_pos = np.arange(event_details_limited.shape[0])
values = list(event_details_limited['value'])

fig = plt.figure(figsize=(12,5))
ax = plt.subplot(1,1,1)
ax.barh(y_pos, event_details_limited['value'], align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(event_details_limited['name'])
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# Add the values in each bar:
for i, v in enumerate(values):
    if v == 0:
        ax.text(0.0005, i, f'{v*100:.2f}%', color='#000000', verticalalignment='center')
    else:
        ax.text(0.0005, i, f'{v*100:.2f}%', color='#FFFFFF', fontweight='bold', verticalalignment='center')
    
plt.title(f'Event detected at {expanded_results.index[0]}', fontsize=12, fontweight='bold')
plt.show()

As we did in the previous notebook, the above bar chart is already of great help to pinpoint what might be going wrong with your asset. Let's load the initial tags description file we prepared in the first notebook and match the sensors with our initial components to group sensors by component:

In [ ]:
# Agregate event diagnostics at the component level:
event_details[['asset', 'sensor']] = event_details['name'].str.split('\\', expand=True)
component_diagnostics = event_details.groupby(by='asset').sum().sort_values(by='value')
component_diagnostics = component_diagnostics[component_diagnostics['value'] > 0.0]

# Prepare Y position and values for bar chart:
y_pos = np.arange(component_diagnostics.shape[0])
values = list(component_diagnostics['value'])

# Plot the bar chart:
fig = plt.figure(figsize=(12,6))
ax = plt.subplot(1,1,1)
ax.barh(y_pos, component_diagnostics['value'], align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(list(component_diagnostics.index))
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# Add the values in each bar:
for i, v in enumerate(values):
    ax.text(0.005, i, f'{v*100:.2f}%', color='#FFFFFF', fontweight='bold', verticalalignment='center')
    
# Show the final plot:
plt.show()

## Inference scheduler operations
---
### Stop inference scheduler
**Be frugal**, running the scheduler is the main cost driver of Amazon Lookout for Equipment. Use the following API to stop an already running inference scheduler. This will stop the periodic inference executions:

In [ ]:
scheduler.stop()

### Start an inference scheduler
You can restart any `STOPPED` inference scheduler using this API:

In [ ]:
scheduler.start()

### Delete an inference scheduler
You can delete a **stopped** scheduler you have no more use of: you can only have one scheduler per model.

In [ ]:
scheduler.stop()
scheduler.delete()

## Conclusion
---
In this notebook, we used the model created in part 3 of this notebook, configured a scheduler and extracted the predictions obtained after it executed a few inferences.